In [1]:
import importlib
import gym_model
importlib.reload(gym_model)
from visualisation import JupyterVizGym, GymDrawer

In [1]:

model_params = {
    "num_trainees": {
        "type": "SliderInt",
        "value": 1,
        "label": "Number of agents:",
        "min": 1,
        "max": 50,
        "step": 1,
    },
    "machine_density": {
        "type": "SliderFloat",
        "value": 0.5,
        "label": "Density of gym equipment:",
        "min": 0,
        "max": 1,
        "step": 0.05,
    },
}

page = JupyterVizGym(
    gym_model.Gym,
    model_params,
    measures=[],
    name="Test gym",
    agent_portrayal=lambda agent: agent.portrayal,
    # space_drawer="default",
    space_drawer=GymDrawer,
)

page

Cannot show ipywidgets in text

[[None <Equipment.CALF_RAISE: 9> <Equipment.LEG_EXTENSION: 7> None
  <Equipment.BICEPS_CURL: 14> None None None]
 [<Equipment.TRICEPS_DIP: 15> <Equipment.CHEST_FLY: 2> None
  <Equipment.ROW: 16> None None None None]
 [<Equipment.LEG_EXTENSION: 7> None <Equipment.LAT_PULLDOWN: 5> None
  <Equipment.HIP_THRUST: 18> None None <Equipment.LEG_PRESS: 6>]
 [None None None <Equipment.REAR_DELT_FLY: 12>
  <Equipment.BICEPS_CURL: 14> <Equipment.REAR_DELT_FLY: 12> None None]
 [<Equipment.TRICEPS_PUSHDOWN: 4> <Equipment.LAT_PULLDOWN: 5>
  <Equipment.SQUAT_RACK: 17> None None None <Equipment.LATERAL_RAISE: 11>
  None]
 [None None <Equipment.TRICEPS_EXTENSION: 3> <Equipment.TRICEPS_DIP: 15>
  <Equipment.SQUAT_RACK: 17> None <Equipment.LEG_CURL: 8>
  <Equipment.BENCH_PRESS: 1>]
 [<Equipment.SHOULDER_PRESS: 10> None <Equipment.LATERAL_RAISE: 11> None
  <Equipment.BENCH_PRESS: 1> <Equipment.SHRUG: 13> None
  <Equipment.LEG_PRESS: 6>]
 [<Equipment.HIP_THRUST: 18> None <Equipment.SHOULDER_PRESS: 10> None
